In [ ]:
# 从数据库读取数据 并 插入es
# 数据库可以是多种数据库

In [12]:
from elasticsearch6 import Elasticsearch
es_client= Elasticsearch("http://118.25.151.8:9200/")

In [16]:
INDEX='test'
INDEX_ALIAS='app.test'
INDEX_BODY='''
{
	"settings": {
		"number_of_shards": 3,
		"number_of_replicas": 2
	},
	"mapping": {
		"_doc": {
			"properties": {
				"user_name": {
					"type": "long"
				},
				"age": {
					"type": "text"
				},
				"passwd": {
					"type": "keyword"
				}
			}
		}
	}
}
'''

In [16]:
# 创建索引
es_client.indices.create(index=INDEX,body=INDEX_BODY)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'test'}

In [18]:
# 判断索引是否存在
es_client.indices.exists(INDEX)

True

# 批量插入数据

In [9]:
# 模拟数据
import uuid
from faker import Faker
fake=Faker()
datas=[]
for i in range(10):
    row={
        "id":uuid.uuid4().hex,
        "user_name":fake.name(),
        "age":fake.random_int(min=18, max=100),
        "passwd":uuid.uuid4().hex
    }
    datas.append(row)
actions=[
    {
        "_index":INDEX,
        "_type":"_doc",
        "_id":data['id'],
        "_source":{k: v for k, v in data.items() if k != "id"}
    } for data in datas
]

In [10]:
actions

[{'_index': 'test',
  '_type': '_doc',
  '_id': '1fa5855fa4584eceaee52676f1c210e3',
  '_source': {'user_name': 'Gene Green',
   'age': 59,
   'passwd': '20e7770fafab49c5b0035a97e06348e4'}},
 {'_index': 'test',
  '_type': '_doc',
  '_id': '712a693ff0f54eef91c43bad754cfc26',
  '_source': {'user_name': 'Mark Smith',
   'age': 78,
   'passwd': '4105db6f5d05409981d6c86888731846'}},
 {'_index': 'test',
  '_type': '_doc',
  '_id': '1263031244e64eefb7016f0999d06003',
  '_source': {'user_name': 'Kelly Chandler',
   'age': 58,
   'passwd': 'b9adcb8901c34d8e82d29b77247798a1'}},
 {'_index': 'test',
  '_type': '_doc',
  '_id': 'dc25df5740404ef7a9751cb310c095cd',
  '_source': {'user_name': 'Erin Torres',
   'age': 46,
   'passwd': '86c6ceae70a14b2195a9ca106befbdbd'}},
 {'_index': 'test',
  '_type': '_doc',
  '_id': '902dfb000afd476caf5f029ebc4bc3e9',
  '_source': {'user_name': 'Sean Bautista',
   'age': 78,
   'passwd': '0a4d0c54603d47edbc7ca673f412e05c'}},
 {'_index': 'test',
  '_type': '_doc',
  '

In [15]:
# 批量插入
from elasticsearch6.helpers import bulk
bulk(es_client,actions)

(10, [])

In [23]:
# 创建别名
es_client.indices.put_alias(INDEX,INDEX_ALIAS)

{'acknowledged': True}

In [24]:
# 获取别名
es_client.indices.get_alias(name=INDEX_ALIAS)

{'test': {'aliases': {'app.test': {}}}}

In [25]:
# 删除别名
es_client.indices.delete_alias(index=INDEX,name=INDEX_ALIAS)

{'acknowledged': True}

In [26]:
# 删除索引
es_client.indices.delete(INDEX)

{'acknowledged': True}